Code adapted from Thomalm GitHub Repository.
Necessary to include unpacker.py

https://github.com/thomalm/svhn-multi-digit

# **Libraries We'll Be Using**

In [0]:
import os
import sys
import tarfile
import numpy as np
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from IPython.display import display, Image, HTML
import h5py

# **Loading SVHN Training and Testing Data  (Format 1: Full-Numbers)**

**Extract Training/Testing images from SVHN train/test.tar.gz files**

In [0]:
def extract_tarball(filename, force=False):
    # Drop the file extension
    root = filename.split('.')[0] 
    
    # If file is already extracted - return
    if os.path.isdir(root) and not force:
        print('%s already present - Skipping extraction of %s.' % (root, filename))
        return
    
    # If file is a tarball file - extract it
    if (filename.endswith("tar.gz")):
        print("Extracting %s ..." % filename)
        tar = tarfile.open(filename, "r:gz")
        tar.extractall()
        tar.close()
        
extract_tarball("test.tar.gz")
extract_tarball("train.tar.gz")

# **Dataset Creation**

**Use Unpacker to get the bounding boxes around the digits in the images**

In [0]:
from unpacker import DigitStructWrapper

def get_bounding_boxes(start_path = '.'):
    """ Extracts a bounding box file and returns a dictionary
    """
    return DigitStructWrapper(start_path).unpack_all()

# Extract the bounding boxes (this will take a while!)
train_bbox = get_bounding_boxes('train/digitStruct.mat')
test_bbox = get_bounding_boxes('test/digitStruct.mat')

# Display the information stored about an individual image
print(json.dumps(train_bbox[0], indent=2))

**Show an example image with bounding box**

In [0]:
from PIL import Image, ImageDraw

def display_bounding_boxes(img, bounding_boxes):
    """Displays an image and overlays the bounding boxes
    """
    # Opens and identifies the given image file
    image = Image.open(img)
    
    # Use draw module can be used to annotate the image
    draw = ImageDraw.Draw(image)
    
    for b in bounding_boxes:
        
        # Bounding box rectangle [x0, y0, x1, y1]
        rectangle = [b['left'], b['top'], b['left'] + b['width'], b['top'] + b['height']]
        
        # Draw a rectangle on top of the image
        draw.rectangle(rectangle, outline="blue")
        
    # Return altered image    
    return image


# Select an image and the corresponding boxes
image = 'train/1.png'
image_bounding_boxes = train_bbox[0]['boxes']
     
# Display image with bounding boxes
display_bounding_boxes(image, image_bounding_boxes)

**Create a Pandas dataframe of the image data**

In [0]:
def dict_to_dataframe(image_bounding_boxes, path):
    """ Helper function for flattening the bounding box dictionary
    """
    # Store each bounding box
    boxes = []
    
    # For each set of bounding boxes
    for image in image_bounding_boxes:
        
        # For every bounding box
        for bbox in image['boxes']:
            
            # Store a dict with the file and bounding box info
            boxes.append({
                    'filename': path + image['filename'],
                    'label': bbox['label'],
                    'width': bbox['width'],
                    'height': bbox['height'],
                    'top': bbox['top'],
                    'left': bbox['left']})
            
    # return the data as a DataFrame
    return pd.DataFrame(boxes)


# We store the bounding boxes here
bbox_file = 'bounding_boxes.csv'

if not os.path.isfile(bbox_file):
    
    # Extract every individual bounding box as DataFrame  
    train_df = dict_to_dataframe(train_bbox, 'train/')
    test_df = dict_to_dataframe(test_bbox, 'test/')

    print("Training", train_df.shape)
    print("Test", test_df.shape)
    print('')

    # Concatenate all the information in a single file
    df = pd.concat([train_df, test_df])
    
    print("Combined", df.shape)

    # Write dataframe to csv
    df.to_csv(bbox_file, index=False)

    # Delete the old dataframes
    del train_df, test_df, train_bbox, test_bbox
    
else:
    # Load preprocessed bounding boxes
    df = pd.read_csv(bbox_file)

# Display the first 10 rows of dataframe
df.head()

# **CODE TO CREATE YOLO .txt TILES FOM BBOX INFORMATION**

**Prepare image information for YOLO t.xt files**

In [0]:
#!rm *.txt

# convert dataframe columns into array format
img_name = np.array(df.filename)
height = np.array(df.height)
label = np.array(df.label).astype(int)
left = np.array(df.left)
top = np.array(df.top)
width = np.array(df.width)

x_center = left + (0.5 * width)
y_center = top + (0.5 * height)

            
# change all the labels for 0 from 10 to 0
for i in range(0, len(label)):
  if label[i] == 10:
    label[i] = 0

**Write training .txt information**

In [0]:
!mkdir train_txt

fname = "train_txt/"
fname += img_name[0].strip('train/.png')
fname += '.txt'
curr_img = img_name[0]
f = open(fname, "w")

for i in range(0, 73257):
  
  if (curr_img != img_name[i]):
    f.close()
    fname = "train_txt/"
    fname += img_name[i].strip('train/.png')
    fname += '.txt'
    curr_img = img_name[i]
    f = open(fname, "w")
    
  
  temp_img = Image.open(img_name[i])
  img_width, img_height = temp_img.size
  temp_img.close()
  
  line = str(label[i])
  line += " "
  line += str(x_center[i] / img_width)
  line += " "
  line += str(y_center[i] / img_height)
  line += " "
  line += str(width[i] / img_width)
  line += " "
  line += str(height[i] / img_height)
  line += "\n"
  
  f.write(line)
  
  
f.close()

**Write testing .txt information**

In [0]:
!mkdir test_txt

fname = "test_txt/"
fname += img_name[73257].strip('test/.png')
fname += '.txt'
curr_img = img_name[73257]
f = open(fname, "w")

for i in range(73257, len(img_name)):
  
  if (curr_img != img_name[i]):
    f.close()
    fname = "test_txt/"
    fname += img_name[i].strip('test/.png')
    fname += '.txt'
    curr_img = img_name[i]
    f = open(fname, "w")
    
  
  temp_img = Image.open(img_name[i])
  img_width, img_height = temp_img.size
  temp_img.close()
  
  line = str(label[i])
  line += " "
  line += str(x_center[i] / img_width)
  line += " "
  line += str(y_center[i] / img_height)
  line += " "
  line += str(width[i] / img_width)
  line += " "
  line += str(height[i] / img_height)
  line += "\n"
  
  f.write(line)
  
  
f.close()

!zip -r /content/train_txt.zip /content/train_txt
!zip -r /content/test_txt.zip /content/test_txt

**Make train and test file path directories for the YOLO data folder**

In [0]:
path = "data/obj/train/"
f = open("train.txt", "w")
for i in range(0, 33402):
  
  fname = str(i+1) + '.png'
  line = path + fname + "\n"
  f.write(line)
    
f.close()


path = "data/obj/test/"
f = open("test.txt", "w")
for i in range(0, 13068):
  
  fname = str(i+1) + '.png'
  line = path + fname + "\n"
  f.write(line)
    
f.close()